# Cleaning and Merging Raw Youtube Comment Data
Clean and Merge Raw Comment Data by Videos

## 1. Loading Modules and Raw Data

In [1]:
import os
import numpy as np
import pandas as pd


In [2]:
path = 'data/' 

In [3]:
os.listdir(path)

['.ipynb_checkpoints', 'youtube_luxury_comments.csv']

In [4]:
raw_comments_df = pd.read_csv(path + 'youtube_luxury_comments.csv')

In [5]:
df = raw_comments_df

## 2. Basic Cleaning of the Data

### 2-1. Eyeballing Data

In [6]:
df.head()

,authorDisplayName,textDisplay,videoId,id,publishedAt,updatedAt
0,한별Hanbyul,안녕하세요!\r\n드디어 명품하울 2탄~💸 영상을 들고 왔어요!\r\n요 며칠 몸이...,yjr2Iaj8Ogc,Ugw_TQ6FrU_0YioBflJ4AaABAg,2018-10-04T09:29:10Z,2018-10-04T09:29:10Z
1,이경민,한별님 한별님이 명품 하울 찍을 때 마다 안녕하세요 한별 입니다 오늘은 여러분들...,yjr2Iaj8Ogc,Ugw_TQ6FrU_0YioBflJ4AaABAg.8lyuCzG-Tsx9AOXDnDoBTR,2020-06-27T02:43:54Z,2020-06-27T02:43:54Z
2,샘조,직업이 따로 있으시나요?,yjr2Iaj8Ogc,Ugw_TQ6FrU_0YioBflJ4AaABAg.8lyuCzG-Tsx96LwYPV4XVf,2020-03-18T16:30:37Z,2020-03-18T16:30:37Z
3,minji seong,덕분에 명품에 대해 1도 몰랐었는데 많이 구경하고 가요\r\n특히 중간에 로고 의미...,yjr2Iaj8Ogc,Ugw_TQ6FrU_0YioBflJ4AaABAg.8lyuCzG-Tsx94lxV3vfpjS,2020-02-08T11:15:36Z,2020-02-08T11:15:36Z
4,서현,샤넬가방 진짜 이뻐요 ㅠㅠ,yjr2Iaj8Ogc,Ugw_TQ6FrU_0YioBflJ4AaABAg.8lyuCzG-Tsx94G-kicDAwt,2020-01-26T16:09:09Z,2020-01-26T16:09:09Z


Check if there are NaN Values or Null Values

In [7]:
df[df['textDisplay'] == 'NaN']

,authorDisplayName,textDisplay,videoId,id,publishedAt,updatedAt


In [8]:
df.loc[df['textDisplay'].isnull() != False]

,authorDisplayName,textDisplay,videoId,id,publishedAt,updatedAt


In [9]:
# Check length
len(df), len(df['videoId'].unique()), len(df['authorDisplayName'].unique())

(15120, 23, 11002)

### 2-2. Cleaning Video Author Data
We found a need to delete author's comments.

In [10]:
df['authorDisplayName'].value_counts(sort=True, ascending=False).head(10)
# ex) 로즈하, 아옳이, 한별...

ᄋᄋ           84
로즈하ROSEHA    40
아옳이          20
ᄋᄋᄋ          19
성이름          19
지구젤리         19
유진           19
김민재          18
한별Hanbyul    17
지은           15
Name: authorDisplayName, dtype: int64

#### Get Author Display Names from Youtube

In [19]:
from pytube import YouTube

author_list = list()
video_author_dict = dict()

for video_id in df['videoId'].unique():
    author = YouTube('https://www.youtube.com/watch?v=' + str(video_id)).author
    author_list.append(author)
    video_author_dict[video_id] = author
    
author_list = np.unique(author_list)

In [20]:
author_list = list(np.unique(author_list))

In [11]:
author_list 

#### Clear Comments from Author

In [12]:
for author in author_list:
    df.drop(df.loc[df['authorDisplayName'] == author].index, inplace = True)

In [13]:
df['authorDisplayName'].value_counts(sort=True, ascending=False).head(10)

ᄋᄋ         84
유진         19
지구젤리       19
ᄋᄋᄋ        19
성이름        19
김민재        18
김지은        15
지은         15
hoho ho    14
김민지        13
Name: authorDisplayName, dtype: int64

#### Delete Duplicate Comments from Same Commentor

In [14]:
df.drop_duplicates(subset = ['authorDisplayName', 'textDisplay'])

,authorDisplayName,textDisplay,videoId,id,publishedAt,updatedAt
1,이경민,한별님 한별님이 명품 하울 찍을 때 마다 안녕하세요 한별 입니다 오늘은 여러분들...,yjr2Iaj8Ogc,Ugw_TQ6FrU_0YioBflJ4AaABAg.8lyuCzG-Tsx9AOXDnDoBTR,2020-06-27T02:43:54Z,2020-06-27T02:43:54Z
2,샘조,직업이 따로 있으시나요?,yjr2Iaj8Ogc,Ugw_TQ6FrU_0YioBflJ4AaABAg.8lyuCzG-Tsx96LwYPV4XVf,2020-03-18T16:30:37Z,2020-03-18T16:30:37Z
3,minji seong,덕분에 명품에 대해 1도 몰랐었는데 많이 구경하고 가요\r\n특히 중간에 로고 의미...,yjr2Iaj8Ogc,Ugw_TQ6FrU_0YioBflJ4AaABAg.8lyuCzG-Tsx94lxV3vfpjS,2020-02-08T11:15:36Z,2020-02-08T11:15:36Z
4,서현,샤넬가방 진짜 이뻐요 ㅠㅠ,yjr2Iaj8Ogc,Ugw_TQ6FrU_0YioBflJ4AaABAg.8lyuCzG-Tsx94G-kicDAwt,2020-01-26T16:09:09Z,2020-01-26T16:09:09Z
5,깡뚱,진짜넘나이쁘네요.말도 되게조곤조곤 잘하시는것같아요ㅎㅎ 뭔가 패션을이해하고 사랑하시는...,yjr2Iaj8Ogc,Ugw_TQ6FrU_0YioBflJ4AaABAg.8lyuCzG-Tsx9-pEcM0divf,2019-10-08T11:14:46Z,2019-10-08T11:14:46Z
...,...,...,...,...,...,...
15115,하이하,언니는 진짜 인간자체명품이에요ㅠ,j3Ybe36WTJI,Ugy1de5o2zYAvBXhyJd4AaABAg.99-VwLKoLqf990FnTh628a,2020-05-23T19:58:27Z,2020-05-23T19:58:27Z
15116,김나연,1번째 댯글~!!!,j3Ybe36WTJI,Ugw_pfSQfPBU1PFFCKN4AaABAg,2020-05-23T13:00:11Z,2020-05-23T13:00:11Z
15117,한세린,"꺄앙ㄱ 언니💗💗1빠,,, ㅠ",j3Ybe36WTJI,UgwLCCPakNBzqYvpK9x4AaABAg,2020-05-23T13:00:08Z,2020-05-23T13:01:04Z
15118,purple lee,1빠다,j3Ybe36WTJI,UgzILto-rqaWHLLkhPZ4AaABAg,2020-05-23T13:00:04Z,2020-05-23T13:00:04Z


### 2-3. Aggregate Comments of Video

In [15]:
Merged_df = pd.pivot_table(df, values='textDisplay', index='videoId', aggfunc = lambda x : ''.join(x))

In [34]:
Merged_df = Merged_df.rename(columns={'textDisplay' : 'mergedComments'})

In [17]:
with pd.option_context('display.max_colwidth', None):
    display(Merged_df.iloc[0])

textDisplay    알고리즘에 의해 보게 되었는데 아옳이님 정말 매력적이시네요!!\r\n옷을 정말 사랑하시는 분 같은데 혹시 나중에 생활 팁으로 '옷 세탁 컨텐츠'도 찍어주시면 정말 좋을 것 같아요\r\n영상 잘 보고 있습니다 감사합니다^^여러분 이거 아무나 사지 마세욧..! 이분이어서 어울리는거니까..☆☆  \r\n선남선녀커플 등급♡♡이 집 옷이 이쁘네요.ㅋㅋㅋ거적대기를 입어두이쁠언냐..남이 옷입는게 이렇게 사랑스럽고 예쁠일인가여 ㅠㅅㅠ넘 재밌어요 😍😍😍😍\r\n저 매장 룩 진짜 이쁘다요 🤎🤎민트옷이 넘 잘어울려요,,한국은 사이즈가m까지만 들어오는군요ㅠ하나하나가 다 레전드네 미쳤다리 ㅠ다 잘 어울려오이언니 진쨔 너무 이뿌고 귀엽댴옷은 정말 이쁜데!! 가격은 그리 이쁘지 않아...ㅋㅋ아언니진짜진심너무구ㅏ엽다 미쳣어 너무기여워 신나서 랄랄라하는 공주님같네 ㅠㅠㅠ세상에 옷 입을 맛 나시겠어요 ㅜㅜㅠ 부러워요명품매장이나 저런백화점 매장에서 대놓고 사진촬영 못하지않나요?  광고성이 아닌이상?능력있고 예쁘고 돈 잘벌고 잘생긴 남편 뒀고 ... 와 언니는 정말 최고에여 ... ❤️❤️옷들이 다너무 예쁘고 좋은데 세탁을 어찌  하실찌?ㅜㅜ협챠늊ㄴㅉ진짜 이언니 너무 사랑스럽다...❤️옷 진짜 이쁘다잘어울리는 원피스 진짜 많앗다요!! 저렇게 돈쓰는데 하루가 쟈미없을수가 없죠...(쥬륵)언니 진짜...세상만사 옷이됐든 악세서리가 됐든 다 소화시켜버리는 인간 활명수,,ㅠ\r\n않히 그래서 그런지 귀걸이가 넘 이뻐보이는것,,영상 재탕을 너무 많이해서 그랬는지 영상에서 소개했었던걸로 기억했는데...아니였고..\r\n귀걸이 정보 아시는분 있으면...ㅏ원츄ㅏ원츄D져라 재수없네 다 대출???6:33 미쳤담쳤다 너무 예뻐 미쳤다 언니 뭐야머지 이 부부.. 자꾸 보게 돼 ㄹㅇ 인간마약이시네들하 진짜 언니... 언니 공주 맞지?언니 열정 100분의1만 나눠주세요 😂😂이분 레알 부럽다...안히 언니가 입으니까 옷걸이에 걸려 있을 때랑 느낌 너무 다르잖아 옷이 언니빨 받잖아남편분    아옳

### 2-4. Merge Author-Video and Video-Comments Data

#### Create and Check DataFrame of Video-Author Data

In [23]:
video_author_df = pd.DataFrame(
    {'videoId' : video_author_dict.keys(),
     'videoAuthor': video_author_dict.values()})


In [24]:
for video, author in video_author_dict.items():
    true_false = video in video_author_df.loc[video_author_df['videoAuthor'] == author]['videoId'].tolist()
    if not true_false:
        print('something misassigned')

In [29]:
Merged_df = Merged_df.merge(video_author_df, on='videoId')

#### Now save uncleaned merged comment data.

In [36]:
Merged_df.to_csv(path + 'Uncleaned_Merged_Youtube_Comments.csv')

## 3. Cleaning Comments Data with KONLPY

#### Load Data and Modules

In [37]:
Uncleaned_Merged_df = pd.read_csv(path + 'Uncleaned_Merged_Youtube_Comments.csv')

In [5]:
df = Uncleaned_Merged_df

In [6]:
from konlpy.tag import Okt
from konlpy.utils import pprint

twitter = Okt()

In [7]:
youtube_luxury_postag = []
for data in df.mergedComments :
    if pd.isnull(data) :
        posresult = None
    else :
        posresult = twitter.pos(data)
    youtube_luxury_postag.append(posresult)
    print(len(youtube_luxury_postag), '/', len(df.mergedComments), 'completed')
print('DONE!')

1 / 23 completed
2 / 23 completed
3 / 23 completed
4 / 23 completed
5 / 23 completed
6 / 23 completed
7 / 23 completed
8 / 23 completed
9 / 23 completed
10 / 23 completed
11 / 23 completed
12 / 23 completed
13 / 23 completed
14 / 23 completed
15 / 23 completed
16 / 23 completed
17 / 23 completed
18 / 23 completed
19 / 23 completed
20 / 23 completed
21 / 23 completed
22 / 23 completed
23 / 23 completed
DONE!


In [8]:
import pickle



In [9]:
## Save pickle
with open("youtube_pos_data.pickle","wb") as fw:
    pickle.dump(youtube_luxury_postag, fw)
 



In [ ]:
## Load pickle
with open("youtube_pos_data.pickle","rb") as fr:
    youtube_luxury_postag = pickle.load(fr)

In [41]:
youtube_luxury_postag[0]

[('알고리즘', 'Noun'),
 ('에', 'Josa'),
 ('의해', 'Adjective'),
 ('보게', 'Verb'),
 ('되었는데', 'Verb'),
 ('아옳이님', 'Noun'),
 ('정말', 'Noun'),
 ('매력', 'Noun'),
 ('적이시네요', 'Verb'),
 ('!!', 'Punctuation'),
 ('\r\n', 'Foreign'),
 ('옷', 'Noun'),
 ('을', 'Josa'),
 ('정말', 'Noun'),
 ('사랑', 'Noun'),
 ('하시는', 'Verb'),
 ('분', 'Noun'),
 ('같은데', 'Adjective'),
 ('혹시', 'Noun'),
 ('나중', 'Noun'),
 ('에', 'Josa'),
 ('생활', 'Noun'),
 ('팁', 'Noun'),
 ('으로', 'Josa'),
 ("'", 'Punctuation'),
 ('옷', 'Noun'),
 ('세탁', 'Noun'),
 ('컨텐츠', 'Noun'),
 ("'", 'Punctuation'),
 ('도', 'Noun'),
 ('찍어주시면', 'Verb'),
 ('정말', 'Noun'),
 ('좋을', 'Adjective'),
 ('것', 'Noun'),
 ('같아요', 'Adjective'),
 ('\r\n', 'Foreign'),
 ('영상', 'Noun'),
 ('잘', 'Verb'),
 ('보고', 'Noun'),
 ('있습니다', 'Adjective'),
 ('감사합니다', 'Verb'),
 ('^^', 'Punctuation'),
 ('여러분', 'Noun'),
 ('이', 'Determiner'),
 ('거', 'Noun'),
 ('아무나', 'Noun'),
 ('사지', 'Noun'),
 ('마세', 'Verb'),
 ('욧', 'Noun'),
 ('..!', 'Punctuation'),
 ('이분', 'Noun'),
 ('이어서', 'Josa'),
 ('어울리는거니까', 'Verb'),
 ('..', 

In [42]:
import nltk

ModuleNotFoundError: No module named 'nltk'